In [72]:
# Importing Libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
import time
import re
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import time
# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# Question 1

Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product 
verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [32]:
#connect to the web driver
driver=webdriver.Chrome(r'C:\Users\BCP\Downloads\chromedriver_win32\chromedriver')

In [33]:
#Specifying the url of the webpage
driver.get("https://www.amazon.in/")


In [34]:
#Assigning the "Search_Keys" option to search the Specific Keyword on Amazon
Search_Keys = input("Type the Keyword to Search on Amazon:----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Amazon.in
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@id='twotabsearchtextbox']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@id='nav-search-submit-text']"))).click()

Type the Keyword to Search on Amazon:----> guitar


# Question 2


In the above question, now scrape the following details of each product listed in first 3 pages of your search results and 
save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the
products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating",
"No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case,
if any of the details are missing for any of the product then replace it by “-“.

In [36]:
#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@id='nav-search-submit-text']"))).click()

#Getting the "href" value/URL of the product in first 3 pages
URT1=[]
for i in range(0, 3):
    for i in driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']"):
        URT1.append(i.get_attribute('href'))

    driver.find_element_by_xpath("//li[@class='a-last']").click()
    time.sleep(3)
        
#Lets create required empty lists required for the 
Avg_Ratings=[]
Brand=[]
No_of_Ratings=[]
Price=[]
Return_Exchange=[]
Expected_Delivery=[]
Availability=[]
Other_Details=[]
Product_URL=[]

#loop for all the required data of product
for url in URT1:
    driver.get(url)
    
#To find the Avg_Ratings of the product
    try:                  
        rate=driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']")
        rate.click()                                                      
        rating=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-base']")
        Avg_Ratings.append(rating.text)
        
    except NoSuchElementException   as e:
        Avg_Ratings.append("-")
               
#To find the Brand Name & Details of the product
    try: 
        brand=driver.find_element_by_xpath("//span[@id='productTitle']")
        Brand.append(brand.text)
        
    except NoSuchElementException   as e:
        Brand.append("-")       
        
#To find the Brand Name & Details of the product  
    try: 
        ratings=driver.find_element_by_xpath("//span[@class='a-size-base a-color-secondary']")
        No_of_Ratings.append(ratings.text)
        
    except NoSuchElementException   as e:
        No_of_Ratings.append("-")
        
        
#To find the Price of the product 
    try: 
        price=driver.find_element_by_xpath("//td[@class='a-span12']")
        Price.append(price.text)
        
    except NoSuchElementException   as e:
        Price.append("-")
              
#To find the Return & Exchange policy of the product 
    try: 
        
        ret_exchange=driver.find_element_by_xpath("//div[@class='a-column a-span3 a-text-center icon-container  icon-grid-width']")#locating the brand
        Return_Exchange.append(ret_exchange.text)
        
    except NoSuchElementException   as e:
        Return_Exchange.append("-")
              
#To find the Expected Delivery policy of the product 
    try: 
        
        Delivery=driver.find_element_by_xpath("//div[@id='dynamicDeliveryMessage']")
        Expected_Delivery.append(Delivery.text)
        
    except NoSuchElementException   as e:
        Expected_Delivery.append("-")        
        
#To find the Availability of the product 
    try: 
        
        availability=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-success']")
        Availability.append(availability.text)
        
    except NoSuchElementException   as e:
        Availability.append("-")       
        
#To find the Other Details of the product
    try: 
        
        Details=driver.find_element_by_xpath("//div[@id='feature-bullets']")
        Other_Details.append(Details.text)
        
    except NoSuchElementException   as e:
        Other_Details.append("-")

#Creating the dataframe for the Searched Keyword/Product searched on Amazon

DF=pd.DataFrame({})
DF['Brand']=Brand
DF['No_of_Ratings']=No_of_Ratings
DF['Avg_Ratings']=Avg_Ratings
DF['Price']=Price
DF['Expected_Delivery']=Expected_Delivery
DF['Other Details']=Other_Details


#Creating the dataframe for Product_Vertical
Product_Vertical=pd.DataFrame({})

#Brand Name of the Guitar
Product_Vertical['Brand Name'] = DF['Brand'].str.split(' ').str[0]

#Name Of Product/Guitar mentioned on Amazon
Product_Vertical['Name Of Product'] = DF['Brand'].str.split(' ').str[1:]
Product_Vertical['Name Of Product'] = DF['Brand'].str.replace('[.,(),]', '')

#Average Ratings for the Guitar 
Product_Vertical['Average Rating']=DF['Avg_Ratings'].str.split(' ').str[0]

#No_of_Ratings for the Guitar 
Product_Vertical['No Of Ratings'] = DF['No_of_Ratings'].str.split(' ').str[0]

#Price of the Guitar
Product_Vertical['Price_INR'] = DF['Price'].str.split(' ').str[1:2]

#Return policy of product
Product_Vertical['Return Policy'] = Return_Exchange

#Expected Delivery of product
Product_Vertical['Expected Delivery'] = DF['Expected_Delivery'].str.split(' ').str[2:5].replace('[.,(),]', '')

#Availability of product
Product_Vertical['Product Availability'] = Availability

#Other Details of product
Product_Vertical['Other Details'] = DF['Other Details'].str.replace('\n', ' ')

#URL of product
Product_Vertical['Product URL'] = URT1

print('The DataFrame for Searched product on https://www.amazon.in/ is :')

print("\n")
Product_Vertical

The DataFrame for Searched product on https://www.amazon.in/ is :




,Brand Name,Name Of Product,Average Rating,No Of Ratings,Price_INR,Return Policy,Expected Delivery,Product Availability,Other Details,Product URL
0,Kadence,Kadence Acoustica Series Semi Acoustic Ash Woo...,3.9,Total,[],-,"[Friday,, July, 23]",In stock.,About this item Product Type : Acoustic Guitar...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,Kadence Slowhand Premium Jumbo Semi Acoustic G...,4.3,Total,[],-,"[Friday,, July, 23]",In stock.,About this item QUALITY STRINGS ---The surface...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Jixing,Jixing Acoustic Guitar 38 Inch Cutaway 038C wi...,3.8,Total,[],-,"[Friday,, July, 23]",In stock.,About this item Brand: Jixing .Type: Acoustic ...,https://www.amazon.in/gp/slredirect/picassoRed...
3,GUITAR,GUITAR BRO - COMBO Black Acoustic Guitar for B...,3.9,Total,[],-,"[July, 23, -]",In stock.,About this item GUITAR BRO +20 mins FREE VIDEO...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,Juârez Acoustic Guitar 38 Inch Cutaway 038C wi...,3.9,Total,[],-,"[Friday,, July, 23]",In stock.,"About this item Black Glossy Finish, Number of...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
169,Whitleys,Whitleys Mini Acoustic Guitar 23 Inch 6 String...,-,Total,[],-,"[July, 26, -]",-,About this item Mini 23 inch Wood 6 Strings Ac...,https://www.amazon.in/Whitleys-Acoustic-Beginn...
170,Cort,Cort AD810BKS Standard Dreadnought Guitar Spru...,4.2,Total,[],-,"[Tuesday,, July, 27]",In stock.,About this item Construction: Dovetail neck jo...,https://www.amazon.in/Cort-AD810BKS-Standard-D...
171,EVONTE,"EVONTE 6-String 24"" Acoustic Guitar Kids Toy B...",-,Total,[],-,"[Friday,, July, 23]",In stock.,PROVIDES SCREEN FREE FUN: A junior scale 6 str...,https://www.amazon.in/EVONTE-6-String-Acoustic...
172,Givson,Givson Acoustic Guitar with Bag Red,1,1,[],-,[],,About this item Type: Acoustic Guitars-Basic S...,https://www.amazon.in/Givson-Acoustic-Guitar-B...


# Question 3

Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords 
‘fruits’, ‘cars’ and ‘Machine Learning’.

In [ ]:
#connect to the web driver
driver=webdriver.Chrome(r'C:\Users\BCP\Downloads\chromedriver_win32\chromedriver')
#Specifying the url of the webpage
driver.get("https://images.google.com/")

In [ ]:
#Assigning the "Search_Keys" option to search the Specific Keyword on Google
Search_Keys = input("> Type the Keyword to Search on https://images.google.com/  :----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Google
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='gLFyf gsfi']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@class='z1asCe MZy1Rb']"))).click()


#Getting the "src" image value of the word on Google
URT1=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
        URT1.append(i.get_attribute('src'))
        
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT1.append(i.get_attribute('data-src'))
    
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT1.append(i.get_attribute('alt'))


#Checking for the null value in Searched_Keyword
print('Checking for the null values in Searched_Keyword')
print('> Any null value present :', Searched_Keyword.isnull().values.any())
print('> Sum of all null values :', Searched_Keyword.isnull().sum())

#Dataframe for Fruit_Images
Fruit_Images=[]
Fruit_Images=URT1

#Checking the length
print('> Length of Fruit_Images :', len(Fruit_Images))
print("\n")

In [ ]:
#connect to the web driver
driver=webdriver.Chrome(r'C:\Users\BCP\Downloads\chromedriver_win32\chromedriver')
#Specifying the url of the webpage
driver.get("https://images.google.com/")

In [15]:
#Assigning the "Search_Keys" option to search the Specific Keyword on Google
Search_Keys = input("> Type the Keyword to Search on https://images.google.com/  :----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Google
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='gLFyf gsfi']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@class='z1asCe MZy1Rb']"))).click()


#Getting the "src" image value of the word on Google
URT2=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
        URT2.append(i.get_attribute('src'))
        
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT2.append(i.get_attribute('data-src'))
    
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT2.append(i.get_attribute('alt'))


#Checking for the null value in Searched_Keyword
print('Checking for the null values in Searched_Keyword')
print('> Any null value present :', Searched_Keyword.isnull().values.any())
print('> Sum of all null values :', Searched_Keyword.isnull().sum())

#Dataframe for Fruit_Images
Car_Images=[]
Car_Images=URT2

#Checking the length
print('> Length of Car_Images :', len(Car_Images))
print("\n")

> Type the Keyword to Search on https://images.google.com/  :----> cars
Checking for the null values in Searched_Keyword
> Any null value present : True
> Sum of all null values : Images    49
dtype: int64
> Length of Car_Images : 147




In [17]:
#connect to the web driver
driver=webdriver.Chrome(r'C:\Users\BCP\Downloads\chromedriver_win32\chromedriver')
#Specifying the url of the webpage
driver.get("https://images.google.com/")

In [18]:
#Assigning the "Search_Keys" option to search the Specific Keyword on Google
Search_Keys = input("> Type the Keyword to Search on https://images.google.com/  :----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Google
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='gLFyf gsfi']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@class='z1asCe MZy1Rb']"))).click()


#Getting the "src" image value of the word on Google
URT3=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
        URT3.append(i.get_attribute('src'))
        
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT3.append(i.get_attribute('data-src'))
    
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT3.append(i.get_attribute('alt'))


#Checking for the null value in Searched_Keyword
print('Checking for the null values in Searched_Keyword')
print('> Any null value present :', Searched_Keyword.isnull().values.any())
print('> Sum of all null values :', Searched_Keyword.isnull().sum())

#Dataframe for Fruit_Images
ML_Images=[]
ML_Images=URT3

#Checking the length
print('> Length of Machine_Learning_Images :', len(ML_Images))
print("\n")

> Type the Keyword to Search on https://images.google.com/  :----> machine learning
Checking for the null values in Searched_Keyword
> Any null value present : True
> Sum of all null values : Images    49
dtype: int64
> Length of Machine_Learning_Images : 147




In [20]:
#Putting all in a single dataframe
Images=pd.DataFrame({})
Images['Fruit_Images']=Fruit_Images[0:100]
Images['Car_Images']=Car_Images[0:100]
Images['Machine_Learning_Images']=ML_Images[0:100]
Images

,Fruit_Images,Car_Images,Machine_Learning_Images
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
...,...,...,...
95,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
96,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
97,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
98,Image result for fruits,Image result for cars,Image result for machine learning


# Question 4

Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following
details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”,“RAM”
, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”,
“Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results
in a dataframe and CSV.

In [68]:
#connect to the web driver
driver=webdriver.Chrome(r'C:\Users\BCP\Downloads\chromedriver_win32\chromedriver')
driver.get('https://www.flipkart.com/')

In [69]:
product=input('Enter the Product to be searched ')
driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
time.sleep(3)
driver.find_element_by_xpath('//input[@title="Search for products, brands and more"]').send_keys(str(product))
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(4)

Enter the Product to be searched pixel 4A


In [70]:
urls=[]
brand=[]
name=[]
color=[]
RAM=[]
ROM=[]
prim_cam=[]
sec_cam=[]
display_size=[]
display_res=[]
processor=[]
processor_core=[]
battery=[]
price=[]
links=driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for i in links:
    urls.append(i.get_attribute('href'))
for i in urls:
    driver.get(i)
    time.sleep(3)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
    try:
        title=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split()
        brand.append(title[0])
    except:
        brand.append('-')
    try:
        name.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[3]/td[2]').text)
    except:
        name.append('-')
    try:
        color.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[4]/td[2]').text)
    except:
        color.append('-')
    try:
        RAM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]').text)
    except:
        RAM.append('-')
    try:
        ROM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]').text)
    except:
        ROM.append('-')
    try:
        prim_cam.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]').text)
    except:
        prim_cam.append('-')
    try:
        sec_cam.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]').text)
    except:
        sec_cam.append('-')
    try:
        display_size.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]').text)
    except:
        display_size.append('-')
    try:
        display_res.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]').text)
    except:
        display_res.append('-')
    try:
        processor.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]').text)
    except:
        processor.append('-')
    try:
        processor_core.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[3]/td[2]').text)
    except:
        processor_core.append('-')
    try:
        battery.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr[1]/td[2]').text)
    except:
        battery.append('-')
    try:
        price.append(driver.find_element_by_xpath('//div[@class="dyC4hf"]/div/div/div').text)
    except:
        price.append('-')
    
flip_search=pd.DataFrame({})
flip_search['brand']=brand
flip_search['name']=name
flip_search['color']=color
flip_search['RAM']=RAM
flip_search['ROM']=ROM
flip_search['prim_cam']=prim_cam
flip_search['sec_cam']=sec_cam
flip_search['display_size']=display_size
flip_search['display_res']=display_res
flip_search['processor']=processor
flip_search['processor_core']=processor_core
flip_search['battery']=battery
flip_search['price']=price
flip_search['urls']=urls
flip_search

,brand,name,color,RAM,ROM,prim_cam,sec_cam,display_size,display_res,processor,processor_core,battery,price,urls
0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,69.4 mm,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,Redmi,Redmi 9 Prime,Matte Black,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,499",https://www.flipkart.com/redmi-9-prime-matte-b...
2,Redmi,Redmi 9 Prime,Sunrise Flare,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,499",https://www.flipkart.com/redmi-9-prime-sunrise...
3,Samsung,Galaxy M31,Ocean Blue,6 GB,64 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,9 mm,"₹15,799",https://www.flipkart.com/samsung-galaxy-m31-oc...
4,Redmi,Note 9 Pro Max,Aurora Blue,6 GB,64 GB,64MP + 8MP + 5MP + 2MP,32MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 720G,Octa Core,76.68 mm,"₹14,999",https://www.flipkart.com/redmi-note-9-pro-max-...
5,Redmi,Redmi Note 9 Pro,Glacier White,4 GB,64 GB,48MP + 8MP + 5MP + 2MP,16MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 pixel,Qualcomm® Snapdragon™ 720G,Octa Core,76.68 mm,"₹13,532",https://www.flipkart.com/redmi-note-9-pro-glac...
6,Redmi,Redmi Note 9 Pro,Aurora Blue,6 GB,128 GB,48MP + 8MP + 5MP + 2MP,16MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 pixel,Qualcomm® Snapdragon™ 720G,Octa Core,76.68 mm,"₹18,699",https://www.flipkart.com/redmi-note-9-pro-auro...
7,Redmi,Redmi 9,Carbon Black,4 GB,64 GB,13MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),720 x 1600$$pixel,MediaTek Helio G35,Octa Core,5000 mAh,"₹9,347",https://www.flipkart.com/redmi-9-carbon-black-...
8,Redmi,Redmi 9,Sporty Orange,4 GB,64 GB,13MP + 8MP,8MP Front Camera,16.59 cm (6.53 inch),720 x 1600$$pixel,MediaTek Helio G35,2.3 GHz,5000 mAh,"₹9,364",https://www.flipkart.com/redmi-9-sporty-orange...
9,Redmi,Redmi 9,Sky Blue,4 GB,64 GB,13MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),720 x 1600$$pixel,MediaTek Helio G35,2.3 GHz,5000 mAh,"₹9,468",https://www.flipkart.com/redmi-9-sky-blue-64-g...


In [71]:
# Saving into a csv file
flip_search.to_csv('flip_search.csv')

# Question 5

Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on
google maps.

In [21]:
driver=webdriver.Chrome(r'C:\Users\BCP\Downloads\chromedriver_win32\chromedriver')
driver.get('https://www.google.com/maps/')

In [23]:
# selecting input field 
city = input('Enter City Name : ') 
driver.find_element_by_id('searchboxinput').send_keys(city)
# pressing Enter
driver.find_element_by_id('searchboxinput').send_keys(Keys.ENTER)
time.sleep(10)
#Scraping latitude and Longitude from page URL
lat=driver.current_url.split('/@')[1].split(',')[0]
long=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]
print('Latitude :',lat,'Longitude :',long)

Enter City Name : Jaipur
Latitude : 26.8851417 Longitude : 75.6504708


# Question 6

Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 –
September 20) from trak.in.


In [31]:
#connect to the web driver
driver=webdriver.Chrome(r'C:\Users\BCP\Downloads\chromedriver_win32\chromedriver')
driver.get('https://trak.in/')
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [32]:
# making directories
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []


In [33]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


# Question 7

Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [21]:
driver=webdriver.Chrome(r'C:\Users\BCP\Downloads\chromedriver_win32\chromedriver')
driver.get('https://www.digit.in/')

In [22]:
#Navigating to the link of best gaming laptop
l1 = driver.find_element_by_xpath("//div[@class='listing_container']/ul/li[9]/a").get_attribute('href')
driver.get(l1)

In [23]:
name = []
price = []
OS = []
display = []
processor = []
HDD = []
RAM = []
weight = []
dimension = []
GPU = []

#names
names=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in names:
    name.append(i.text)
    
#os
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)
    
#display
displays=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)
    
#processor
processors=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in processors:
    processor.append(i.text)
processor

#memory
memories=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")#list of specificaion name
memories_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")#values of specifiations 
for i in range(len(memories)):
        if memories[i].text=='Memory':
            HDD.append(memories_spec[i].text.split('/')[0])
            RAM.append(memories_spec[i].text.split('/')[1])
        else:
            HDD.append('No details available')#append no details as value for memory is missing in some of the laptops
            RAM.append('No details available')#append no details as value for memory is missing in some of the laptops

#weight
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
weight_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(weights)):
        if weights[i].text=='Weight':
            weight.append(weight_spec[i].text)
        
#dimension
dimension=[]
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
dims_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(dims)):
        if dims[i].text=='Dimension':
            dimension.append(dims_spec[i].text)

#graphical processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
GPUs_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(GPUs)):
        if GPUs[i].text=='Graphics Processor':
            GPU.append(GPUs_spec[i].text)
        
full_specs=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")#getting the url of full specs links
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
    
for i in full_specs:#iterating throug every laptops full specs' page
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(prices.text)
    except NoSuchElementException:#exception handling for no price details
        price.append("No details available")
        
df=pd.DataFrame({"Name":name,
                "Price":price,
                "OS":OS,
                "Display":display,
                "HDD":HDD,
                 "RAM":RAM,
                "processor":processor,
                "weight":weight,
                "Dimension":dimension,
                "Graphical processor":GPU})
df

,Name,Price,OS,Display,HDD,RAM,processor,weight,Dimension,Graphical processor
0,MSI GT76 TITAN DT 9SG,"379,990",WINDOWS 10 PRO,"17.3"" (UHD 3840X2160)",1 TB HDD,64 GBGB DDR4,INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080
1,ALIENWARE 17 AREA-51M,"489,909",WINDOWS 10 PRO,"17.3"" (FHD (1920 X 1080))",1 TB PCIe SSD,32GB DDR4,INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080
2,HP OMEN 15 2020,"114,990",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,16 GBGB DDR4,INTEL I7-10750H 10TH GEN | 1.6GHZ,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti
3,ASUS ZEPHYRUS G14,"164,990",WINDOWS 10 HOME,"14"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060
4,LENOVO LEGION Y540,"79,990",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",1 TB SSD,8GB DDR4,9TH GENERATION CORE INTEL I7-9750H | 2.6 GHZ,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060
5,ASUS ROG ZEPHYRUS G GA502,"79,990",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,16GB DDR4,AMD RYZEN 7 QUAD CORE 3750H | 2.3 GHZ,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti
6,ASUS ROG ZEPHYRUS S GX531,"146,000",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512GB SSD,16 GBGB DDR4,9TH GEN INTEL CORE I7-8750H | 2.2 GHZ,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q)
7,MSI GT83VR 7RE TITAN SLI,"349,990",WINDOWS 10 HOME 64 BIT,"18.4"" (1920 X 1080)",1.5 TB SATA,64GB DDR4,INTEL CM238 CORE I7-7820HK+CM238 7TH GEN | 3.5GHZ,5.5,458 x 339 x 69,Dual GTX1070
8,ASUS ROG ZEPHYRUS DUO 15,"259,990",WINDOWS 10,"15.6"" (3840 X 1100)",512 GB SSD,4 GBGB DDR4,INTEL CORE I7 10TH GEN 10875H | NA,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q
9,DELL G3,"61,990",WINDOWS 10 HOME PLUS,15.6 MP | NA,256 + 1 TB SSD + SATA,16GB DDR4,8TH GENERATION INTEL® CORE™ I7-8750H | NA,2.53,22.7 X 380 X 258,NVIDIA® GeForce® GTX


# Question 8

Write a python program to scrape the details for all billionaires from www.forbes.com.

In [55]:
# Activating the chrome browser
driver=webdriver.Chrome(r'C:\Users\BCP\Downloads\chromedriver_win32\chromedriver') 

In [56]:
# Opening the forbes.com
url = "https://www.forbes.com/?sh=69e6b8c92254"
driver.get(url)

In [57]:
#clicking the explore button
button = driver.find_element_by_xpath("//button[@class='icon--hamburger']")
button.click()

In [58]:
#select billioners 
billioners = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
billioners.click()

In [59]:
#select world billioners 
world_billioners= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_billioners.click()

In [60]:
#make empty listes
Rank = [] 
Person_Name = [] 
total_net_worth = [] 
Age = []
country_of_citizenship = [] 
Source = []
industry = []

In [61]:
while(True):
    #scraping rank of billionaire
    rank= driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rank:
        Rank.append(i.text)
    
    #scraping name of billionaire
    name= driver.find_elements_by_xpath("//div[@class='personName']//div")
    for i in name:
        Person_Name.append(i.text)
    
     #scraping Age of billionaire
    age= driver.find_elements_by_xpath("//div[@class='age']//div")
    for i in age:
        Age.append(i.text)   
    
    #scraping citizenship of billionaire    
    citizenship= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in citizenship:
        country_of_citizenship.append(i.text)
    
    #scraping source of income
    source= driver.find_elements_by_xpath("//div[@class='source']")
    for i in source:
        Source.append(i.text)    
        
    #scraping Age of billionaire
    industries= driver.find_elements_by_xpath("//div[@class='category']//div")
    for i in industries:
        industry.append(i.text)
        
    #scraping net_worth of billionaire
    net_worth= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for i in net_worth:
        total_net_worth.append(i.text)
        
    try:
        next_button = driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break  

In [62]:
Net_Worth = []        
for i in range(0,len(total_net_worth),2):
        Net_Worth.append(total_net_worth[i])

In [63]:
  #creating dataframe
df=pd.DataFrame({'Rank':Rank,
                'Name':Person_Name,
                'Net Worth':Net_Worth,
                'Age':Age,
                'Country of Citizenship':country_of_citizenship,
                'Source':Source,
                'Industry':industry})
df

,Rank,Name,Net Worth,Age,Country of Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


# Question 9

Write a program to extract at least 500 Comments, Comment upvote and time when comment
was posted from any YouTube Video.

In [102]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

# Opening the youtube.com
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

#finding element for search bar
search_bar = driver.find_element_by_id('search')
search_bar.send_keys("GOT")  #entering Video name
time.sleep(1)

#clicking on search button
search_btn = driver.find_element_by_id("search-icon-legacy")  
search_btn.click()
time.sleep(1)

#clicking on first video
link_click = driver.find_element_by_xpath("//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()
time.sleep(1)

In [103]:
# 1000 time we scroll down by 10000 in order to generate more Comments
for _ in range(1000):
    driver.execute_script("window.scrollBy(0,10000)")

In [104]:
#make empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

In [105]:
#scrape comments
cm = driver.find_elements_by_id("content-text")
for i in cm:
    if i.text is None:
        comments.append("--")
    else:
        comments.append(i.text)

In [106]:
# scrape time when comment was posted
tm = driver.find_elements_by_xpath("//a[contains(text(),'ago')]")
for i in tm:
    Time.append(i.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])

In [107]:
# scrape the comment likes
like = driver.find_elements_by_xpath("//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for i in like:
    Likes.append(i.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])

In [108]:
#creating dataframe
df=pd.DataFrame({'Comments':comments,
                'Time':comment_time,
                'Likes':No_of_Likes})
df

,Comments,Time,Likes


# Question 10

Write a python program to scrape a data for all available Hostels from
https://www.hostelworld.com/ in “London” location. You have to scrape hostel name,
distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms
from price, facilities and property description.

In [95]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)

# Opening the homepage of hostelworld.com
url = "https://www.hostelworld.com/"
driver.get(url)

In [ ]:
#locating the location search bar
search_loc = driver.find_element_by_id('search-input-field')
# write Lonodn in search bar
search_loc.send_keys("London")
time.sleep(2)

#select london
london = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div')
london.click()
time.sleep(2)


# do click in search button
search_btn = driver.find_element_by_id('search-button')
search_btn.click()

In [97]:
# Make empty lists
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []

In [98]:
while(True):
    
    # Hostel name
    names = driver.find_elements_by_xpath('//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    # Distance from city centre
    dis = driver.find_elements_by_xpath('//span[@class="description"]')
    for d in dis:
        Distance.append(d.text)
    time.sleep(2)
        
    # Overall Review    
    review = driver.find_elements_by_xpath('//div[@class="keyword"]//span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    # Total No of reviews     
    t_review = driver.find_elements_by_xpath('//div[@class="reviews"]')
    for tr in t_review:
        total_reviews.append(tr.text)
    time.sleep(2)
    
    # facilities
    service = driver.find_elements_by_xpath('//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    # Prices    
    prices = driver.find_elements_by_xpath('//div[@class="price-col"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
        
    try:
        next_button = driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break

In [99]:
# Separate  Privates_From price  and  Dorms_From price
Privates_From = []
for i in range(0,len(price),2):
    Privates_From.append(price[i])

Dorms_From = []
for i in range(1,len(price),2):
    Dorms_From.append(price[i])

In [100]:
# scrape Hostels URL
hostel_url = []

while(True):
    urls = driver.find_elements_by_xpath('//h2[@class="title title-6"]//a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break


        
Rate = []
for page in hostel_url:
    driver.get(page)
    
    # Rating
    try:
        ratings = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    
    # Property Description
    try:
        pd = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  

    
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0])

In [101]:
#creating dataframe
df = pd.DataFrame({'Hostel Name':Hostel_Name,
                'Distance from city centre':Distance,
                'Overall Review':overall_review[3:],
                'Total Reviews':total_reviews,
                'Facilities':facilities,
                'Privates From Price':Privates_From,
                'Dorms From Price':Dorms_From,
                'Rating':Rating,
                'Property Description':property_description})
df

,Hostel Name,Distance from city centre,Overall Review,Total Reviews,Facilities,Privates From Price,Dorms From Price,Rating,Property Description
